<a href="https://colab.research.google.com/github/HARSHGit45/Deep-Learning-Neural-Networks/blob/main/Practical_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import canny
from skimage import morphology, segmentation
from skimage.color import label2rgb
from scipy.ndimage import binary_fill_holes, sobel, label

import kagglehub
path = kagglehub.dataset_download("lincolnzh/martianlunar-crater-detection-dataset")
print("Path to dataset files:", path)
import os




Path to dataset files: /root/.cache/kagglehub/datasets/lincolnzh/martianlunar-crater-detection-dataset/versions/2


In [2]:
class ThresholdingSegmentation:
    def __init__(self, image_path):
        self.image_path = image_path
        self.image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

    def binary_threshold(self, threshold_value=127):
        ret, thresh1 = cv.threshold(self.image, threshold_value, 255, cv.THRESH_BINARY)
        plt.imshow(thresh1, cmap='gray', vmin=0, vmax=255)
        plt.title('Binary Thresholding')
        plt.show()

    def adaptive_threshold(self):
        th2 = cv.adaptiveThreshold(self.image, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 2)
        plt.imshow(th2, cmap='gray', vmin=0, vmax=255)
        plt.title('Adaptive Thresholding')
        plt.show()

    def otsu_threshold(self):
        ret2, th3 = cv.threshold(self.image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        plt.imshow(th3, cmap='gray', vmin=0, vmax=255)
        plt.title('Otsu’s Thresholding')
        plt.show()


In [3]:
class RegionGrowingSegmentation:
    def __init__(self, image_path):
        self.image_path = image_path
        self.image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

    def canny_edge_detection(self):
        edges = canny(self.image / 255.0)
        plt.imshow(edges, cmap='gray')
        plt.title('Canny Edge Detection')
        plt.show()
        return edges

    def region_fill(self, edges):
        fill_im = binary_fill_holes(edges)
        plt.imshow(fill_im, cmap='gray')
        plt.title('Region Filling')
        plt.show()
        return fill_im

    def watershed_segmentation(self):
        elevation_map = sobel(self.image / 255.0)
        plt.imshow(elevation_map, cmap='gray')
        plt.title('Elevation Map')
        plt.show()

        markers = np.zeros_like(self.image)
        markers[self.image < 30] = 1
        markers[self.image > 150] = 2
        plt.imshow(markers, cmap='gray')
        plt.title('Markers for Watershed')
        plt.show()

        segment = segmentation.watershed(elevation_map, markers)
        segment = binary_fill_holes(segment - 1)
        label_rock, _ = label(segment)
        image_label_overlay = label2rgb(label_rock, image=self.image / 255.0)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
        ax1.imshow(self.image, cmap='gray')
        ax1.contour(segment, [0.8], linewidths=1.8, colors='w')
        ax1.set_title('Contour Overlay')
        ax2.imshow(image_label_overlay)
        ax2.set_title('Watershed Segmentation')
        plt.show()


In [4]:
def main_menu():
    image_path = '/root/.cache/kagglehub/datasets/lincolnzh/martianlunar-crater-detection-dataset/versions/2/craters/train/images/mars_crater--24-_jpg.rf.4614bb6844fda70ea8a3b4f515cb7e8f.jpg'
    while True:
        print("\n--- Segmentation Menu ---")
        print("1. Thresholding Segmentation")
        print("2. Region Growing Segmentation")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == '1':
            thresholding = ThresholdingSegmentation(image_path)
            print("\n--- Thresholding Options ---")
            print("1. Binary Threshold")
            print("2. Adaptive Threshold")
            print("3. Otsu's Threshold")
            sub_choice = input("Enter your choice (1-3): ")

            if sub_choice == '1':
                thresholding.binary_threshold()
            elif sub_choice == '2':
                thresholding.adaptive_threshold()
            elif sub_choice == '3':
                thresholding.otsu_threshold()
            else:
                print("Invalid option.")

        elif choice == '2':
            region_growing = RegionGrowingSegmentation(image_path)
            edges = region_growing.canny_edge_detection()
            filled = region_growing.region_fill(edges)
            region_growing.watershed_segmentation()

        elif choice == '3':
            print("Exiting the program.")
            break

        else:
            print("Invalid option. Please try again.")



In [5]:
#main_menu()